In [1]:
#RANDOM FOREST
import cv2,os

#Este bloque de código genera un diccionario de python con índice como valores y nombres de las carpetas como claves.

ubidata='dataset'
grupos=os.listdir(ubidata)
identifier=[i for i in range(len(grupos))]

groupandID=dict(zip(grupos,identifier))

print(identifier)
print(grupos)
print(groupandID)

[0, 1]
['with mask', 'without mask']
{'with mask': 0, 'without mask': 1}


In [2]:
data=[]
target=[]
tamañoimagen=100


for category in grupos:
    ubicarpetag=os.path.join(ubidata,category)
    listanomimagenes=os.listdir(ubicarpetag)
        
    for nombreimagen in listanomimagenes:
        ubiimagen=os.path.join(ubicarpetag,nombreimagen)
        imagen=cv2.imread(ubiimagen)

        try:
            #Aqui se convierte la imagen a escala de grises
            imgenbn=cv2.cvtColor(imagen,cv2.COLOR_BGR2GRAY)           
            #Aqui cambiamos el tamaño de la escala de grises a 100x100
            #necesitamos un tamaño común fijo para todas las imágenes en el conjunto de datos
            resized=cv2.resize(imgenbn,(tamañoimagen,tamañoimagen))
            data.append(resized)
            target.append(groupandID[category])
            #juntamos la imagen y la etiqueta (categorizada) en la lista (conjunto de datos)

        except Exception as e:
            print('Exception:',e)
            #si ocurre alguna excepción, la excepción se imprimirá aquí. Y pasa a la siguiente imagen.

In [3]:
import numpy as np

target=np.array(target)
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],tamañoimagen,tamañoimagen,1))

from keras.utils import np_utils

new_target=np_utils.to_categorical(target)

In [4]:
np.save('data',data)
np.save('target',new_target)

In [5]:
import numpy as np

data=np.load('data.npy')
target=np.load('target.npy')

In [6]:
from sklearn.ensemble import RandomForestClassifier

In [7]:
model=RandomForestClassifier()

In [8]:
from sklearn.model_selection import train_test_split

xdata,ydata,xtarget,ytarget=train_test_split(data,target,test_size=0.1)

In [9]:
#sklearn expects i/p to be 2d array-model.fit(x_train,y_train)=>reshape to 2d array
nsamples, nx, ny, nrgb = xdata.shape
x_train2 = xdata.reshape((nsamples,nx*ny*nrgb))
nsamples, nx, ny, nrgb = ydata.shape
x_target2 = ydata.reshape((nsamples,nx*ny*nrgb))
model.fit(x_train2,xtarget)

RandomForestClassifier()

In [10]:
y_pred=model.predict(x_target2)

In [11]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
accuracy_score(y_pred,ytarget)
print(classification_report(y_pred,ytarget))

              precision    recall  f1-score   support

           0       0.93      0.96      0.95        27
           1       0.98      0.96      0.97        50

   micro avg       0.96      0.96      0.96        77
   macro avg       0.95      0.96      0.96        77
weighted avg       0.96      0.96      0.96        77
 samples avg       0.96      0.96      0.96        77

